In [ ]:
# Automatic1111 Server
automatic1111_server = "http://127.0.0.1:7777"
automatic1111_model = "avalonTruvision_v4"

# ollama stuff. Model can be llava or bakllava
ollama_server = "http://127.0.0.1:11434"
ollama_model = "bakllava"

# Number of images to auto generate
count = 3

In [ ]:
"""
    Generate an image from a text prompt
    
Args:
    prompt (str): A prompt detailing the request

Returns:
    An image representng the prompt
"""

import requests
import base64
from IPython.display import Image
import datetime
import os

def generate_image(prompt: str = "Cute cat", neg_prompt: str = ""):
    try:
        # Define the URL and the payload to send.
        url = automatic1111_server

        # Use the docs and add variables to payload as needed, this is basic
        # http://127.0.0.1:7860/docs#/default/text2imgapi_sdapi_v1_txt2img_post
        payload = {
            "prompt": prompt,
            "negative_prompt": neg_prompt,
            "steps": 26,
            "width": 400,
            "height": 400,
            "tiling": False,
            "override_settings": {
                "sd_model_checkpoint": automatic1111_model,
                "CLIP_stop_at_last_layers": 2,
            }
        }

        # Send said payload to said URL through the API.
        response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)
        response.raise_for_status()  # Raise an exception for non-200 status codes

        # Get current date and time # Format current date and time to DDMMYYHHMM
        current_datetime = datetime.datetime.now()
        formatted_datetime = current_datetime.strftime("%d%m%y%H%M")

        # Decode and save the image.
        with open(f'tmp/{formatted_datetime}.png', 'wb') as f:
            f.write(base64.b64decode(response.json()['images'][0]))
        return f'tmp/{formatted_datetime}.png'

    except requests.RequestException as e:
        raise Exception(f"Failed to generate image: {str(e)}")
    
    except Exception as e:
        raise Exception(f"An error occurred while generating image: {str(e)}")


In [ ]:
import requests
import json
import base64
from urllib.parse import urlparse
import os

def summarize_image(image_path: str, prompt: str = "Describe this image in a single sentence"):
    """
    Summarizes the contents of an image.

    Args:
        image_path (str): The local path or URL of the image.
        prompt (str): A prompt detailing the request.

    Returns:
        str: A text summary of the image contents.
    """
    try:
        # Function to download image from URL
        def download_image(image_url: str) -> str:
            response = requests.get(image_url)
            response.raise_for_status()  # Raise an exception for non-200 status codes
            
            # Parse the URL to extract the filename
            parsed_url = urlparse(image_url)
            filename = os.path.basename(parsed_url.path)
            # Save the image locally
            local_path = f".tmp/{filename}"
            with open(local_path, "wb") as file:
                file.write(response.content)
            return local_path
        
        # Check if the image_path is a URL
        if urlparse(image_path).scheme:
            # If it's a URL, download the image
            image_path = download_image(image_path)

        # Encode image to base64
        with open(image_path, "rb") as image_file:
            image_base64 = base64.b64encode(image_file.read()).decode('utf-8')

        url = f"{ollama_server}/api/generate"
        payload = {
            "model": ollama_model,
            "prompt": prompt,
            "stream": False,
            "images": [image_base64]
        }
        response = requests.post(url, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for non-200 status codes
        
        # Parse JSON response
        json_response = response.json()
        # Access the value of the "response" key
        response_value = json_response["response"]
        return response_value
    
    except requests.RequestException as e:
        raise Exception(f"Failed to summarize image: {str(e)}")

    except Exception as e:
        raise Exception(f"An error occurred while summarizing image: {str(e)}")


In [ ]:
import time

prompt =  """Cute colorful unicorn, rainbow, vector """ 

print(prompt)

persistent_prompt = """Neo-geo, neo-expressionism, pop-art. Minimalism, white background"""
prompt =  persistent_prompt + prompt

neg_prompt = """photo, realistic, photorealism"""
print(f"Negative Prompt: {neg_prompt}{chr(10)}")

for i in range(0, count):
    # Time image generartion
    start_time = time.time()    
    
    image_path = generate_image(prompt, neg_prompt)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    img = Image(image_path)
    display(img)
    
    print(f"Image generated in {execution_time:.2f} seconds") 
    print(f'FilePath: {image_path}{chr(10)}')
    
    # Time llava
    start_time = time.time()    

    prompt = summarize_image(image_path)

    end_time = time.time()
    execution_time = end_time - start_time
    print(prompt)
    print(f'Summary took {execution_time:.2f} seconds{chr(10)}')    
    
    prompt = persistent_prompt + prompt